In [1]:
Python 3 (ipykernel)


SyntaxError: invalid syntax (2207384396.py, line 1)

In [ ]:
import socket
import threading
from Crypto.Cipher import AES
import base64

KEY = b'16byteslongkey!!'  # 16 bytes AES key

def encrypt(message):
    cipher = AES.new(KEY, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(message.encode())
    return base64.b64encode(cipher.nonce + tag + ciphertext).decode()

def decrypt(data):
    data = base64.b64decode(data)
    nonce = data[:16]
    tag = data[16:32]
    ciphertext = data[32:]
    cipher = AES.new(KEY, AES.MODE_EAX, nonce=nonce)
    return cipher.decrypt_and_verify(ciphertext, tag).decode()

clients = []

def handle_client(conn, addr):
    print(f"[+] Connected: {addr}")
    while True:
        try:
            encrypted_msg = conn.recv(4096).decode()
            if not encrypted_msg:
                break

            message = decrypt(encrypted_msg)
            print(f"{addr}: {message}")

            # broadcast encrypted
            for c in clients:
                if c != conn:
                    c.send(encrypt(message).encode())

        except:
            break

    conn.close()
    clients.remove(conn)

def start_server():
    server = socket.socket()
    server.bind(("0.0.0.0", 8080))
    server.listen()

    print("🔥 Server started on port 8080...")
    print("Waiting for clients to connect...")

    while True:
        conn, addr = server.accept()
        clients.append(conn)
        threading.Thread(target=handle_client, args=(conn, addr)).start()

start_server()


🔥 Server started on port 8080...
Waiting for clients to connect...
[+] Connected: ('127.0.0.1', 61786)
[+] Connected: ('127.0.0.1', 51407)
('127.0.0.1', 51407): Hello everyone!
('127.0.0.1', 51407): How are you?
